# Let's try to optimize the classifier

In [1]:
# # Required to preprocess text for BERT inputs
!pip install -q tensorflow-text==2.11.*
!pip install -q tf-models-official==2.11.0
!pip install tf-models-official

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 55.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 85.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 77.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 93.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 kB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 114.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-datasets 4.9.2 requires protobuf>=3.20, but you have protobuf 3.19.6 which is incompatible.
tensorflow-metadata 1.13.1 requires protobuf<5,>=3.20.3, but you have protobuf 3.19.6 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 M

In [ ]:
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization

tf.get_logger().setLevel('ERROR')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd

df = pd.read_csv("drive/My Drive/BNGKT - Travelah CHATBOT/Intent/intents.csv")
df.head(10)

,utterance,intent,response,alias
0,what is bali known for?,bali_general_faq,Hey buddy! Bali is totally the place to be wit...,Tell me about Bali
1,what's the lowdown on what makes bali so dang ...,bali_general_faq,"Hey, my friend! Bali is absolutely the place t...",NaN
2,what's the deal with bali's claim to fame?,bali_general_faq,"Hey there, buddy! Bali is hands down the place...",NaN
3,"why is bali the talk of the town, my friend?",bali_general_faq,"Hey, buddy! Bali is the ultimate destination t...",NaN
4,"what's the scoop on bali's reputation, yo?",bali_general_faq,"Hey, my friend! Bali is the perfect spot to be...",NaN
5,can you spill the beans on what bali is all ab...,bali_general_faq,"Hey there, buddy! Bali is the go-to place with...",NaN
6,"what's the big fuss about bali, anyway?",bali_general_faq,NaN,NaN
7,why is bali considered a hotshot destination?,bali_general_faq,NaN,NaN
8,what sets bali apart from the rest of the pack...,bali_general_faq,NaN,NaN
9,what's the buzz about bali's standout features?,bali_general_faq,NaN,NaN


Train, val, test split

In [4]:
# converting type of columns to 'category'
df['intent'] = df['intent'].astype('category')
# Assigning numerical values and storing in another column
df['intent_encoding'] = df['intent'].cat.codes
df

,utterance,intent,response,alias,intent_encoding
0,what is bali known for?,bali_general_faq,Hey buddy! Bali is totally the place to be wit...,Tell me about Bali,0
1,what's the lowdown on what makes bali so dang ...,bali_general_faq,"Hey, my friend! Bali is absolutely the place t...",NaN,0
2,what's the deal with bali's claim to fame?,bali_general_faq,"Hey there, buddy! Bali is hands down the place...",NaN,0
3,"why is bali the talk of the town, my friend?",bali_general_faq,"Hey, buddy! Bali is the ultimate destination t...",NaN,0
4,"what's the scoop on bali's reputation, yo?",bali_general_faq,"Hey, my friend! Bali is the perfect spot to be...",NaN,0
...,...,...,...,...,...
4412,"what's the weather game in bali, my man?",weather_faq,NaN,NaN,20
4413,"how's the weather action in bali, bro?",weather_faq,NaN,NaN,20
4414,"give me the lowdown on bali's weather scene, h...",weather_faq,NaN,NaN,20
4415,"what's the weather vibe like in bali, dude?",weather_faq,NaN,NaN,20


In [5]:
import json
import numpy as np

df['intent_encoding'] = df['intent_encoding'].astype(str)
data = {"intent": []}
for intent, group in df.groupby('intent'):
    intent_data = {
        "intent": intent,
        "patterns": group['utterance'].tolist(),
        "responses": group['response'].dropna().unique().tolist(),
        "intent_encoding": group['intent_encoding'].iloc[0],
        "alias": group['alias'].iloc[0]
    }
    data["intent"].append(intent_data)

# Convert NaN values to None
def nan_to_none(obj):
    if isinstance(obj, float) and np.isnan(obj):
        return None
    return obj

json_data = json.dumps(data, indent=4, default=nan_to_none)

with open("intents.json", "w") as outfile:
    outfile.write(json_data)

In [6]:
from sklearn.model_selection import train_test_split
X = df['utterance']
y = df['intent_encoding']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

X_train = X_train.values
X_val = X_val.values
X_test = X_test.values
y_train = y_train.values
y_val = y_val.values
y_test = y_test.values

In [7]:
num_classes = len(df['intent'].unique())
y_train_one_hot = tf.one_hot(y_train, num_classes)
y_val_one_hot = tf.one_hot(y_val, num_classes)
y_test_one_hot = tf.one_hot(y_test, num_classes)

In [8]:
bert_encoder_model = 'https://tfhub.dev/tensorflow/albert_en_base/3'
bert_preprocessor = 'https://tfhub.dev/tensorflow/albert_en_preprocess/3'

In [9]:
bert_preprocess_model = hub.KerasLayer(bert_preprocessor)
bert_model = hub.KerasLayer(bert_encoder_model)

In [10]:
def build_smallBERT_CNN_classifier_model():
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
    preprocessing_layer = hub.KerasLayer(bert_preprocessor, name='preprocessing')
    encoder_inputs = preprocessing_layer(text_input)
    encoder = hub.KerasLayer(bert_encoder_model, trainable=True, name='BERT_encoder')
    outputs = encoder(encoder_inputs)
    net = sequence_output = outputs["sequence_output"]

    net = tf.keras.layers.Conv1D(32, 2, activation='relu')(net)
    net = tf.keras.layers.Conv1D(64, 2, activation='relu')(net)
    net = tf.keras.layers.MaxPooling1D(pool_size=2)(net)
    
    net = tf.keras.layers.GlobalMaxPool1D()(net)
    
    net = tf.keras.layers.Dense(512, activation="relu")(net)
    net = tf.keras.layers.Dropout(0.1)(net)
    
    net = tf.keras.layers.Dense(num_classes, activation="softmax", name='classifier')(net)
    
    return tf.keras.Model(text_input, net)

In [ ]:
def build_AlBERT_LSTM_classifier_model():
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
    preprocessing_layer = hub.KerasLayer(bert_preprocessor, name='preprocessing')
    encoder_inputs = preprocessing_layer(text_input)
    encoder = hub.KerasLayer(bert_encoder_model, trainable=True, name='BERT_encoder')
    outputs = encoder(encoder_inputs)
    sequence_output = outputs["sequence_output"]
    
    # Add an LSTM layer after BERT
    lstm_output = tf.keras.layers.LSTM(64)(sequence_output)
    
    net = tf.keras.layers.Dense(512, activation="relu")(lstm_output)
    net = tf.keras.layers.Dropout(0.1)(net)
    
    net = tf.keras.layers.Dense(num_classes, activation="softmax", name='classifier')(net)
    
    return tf.keras.Model(text_input, net)

Build Model

In [11]:
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
metrics = tf.metrics.CategoricalAccuracy()
epochs = 15
optimizer=tf.keras.optimizers.Adam(1e-5)

classifier_smallBERT_model_cnn = build_smallBERT_CNN_classifier_model()

In [12]:
classifier_smallBERT_model_cnn.compile(optimizer=optimizer,
                         loss=loss,
                         metrics=metrics)

Train

In [14]:
class MyCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs['val_categorical_accuracy'] >= 0.94:
            print("\nValidation accuracy reached 95%!")
            self.model.stop_training = True

In [15]:
callback = MyCallback()

history = classifier_smallBERT_model_cnn.fit(x=X_train, y=y_train_one_hot,
                                             validation_data=(X_val, y_val_one_hot),
                                             batch_size=32,
                                             epochs=epochs,
                                             callbacks=[callback])

Epoch 1/15
89/89 [==============================] - 136s 1s/step - loss: 3.0010 - categorical_accuracy: 0.0892 - val_loss: 2.8183 - val_categorical_accuracy: 0.1391
Epoch 2/15
89/89 [==============================] - 108s 1s/step - loss: 2.7490 - categorical_accuracy: 0.1476 - val_loss: 2.5960 - val_categorical_accuracy: 0.2217
Epoch 3/15
89/89 [==============================] - 108s 1s/step - loss: 2.5044 - categorical_accuracy: 0.2530 - val_loss: 2.3288 - val_categorical_accuracy: 0.3767
Epoch 4/15
89/89 [==============================] - 108s 1s/step - loss: 2.2052 - categorical_accuracy: 0.3808 - val_loss: 2.0036 - val_categorical_accuracy: 0.4819
Epoch 5/15
89/89 [==============================] - 108s 1s/step - loss: 1.8486 - categorical_accuracy: 0.5007 - val_loss: 1.6513 - val_categorical_accuracy: 0.5566
Epoch 6/15
89/89 [==============================] - 108s 1s/step - loss: 1.4745 - categorical_accuracy: 0.6044 - val_loss: 1.2993 - val_categorical_accuracy: 0.6550
Epoch 7/15

Evaluate

In [16]:
loss, accuracy = classifier_smallBERT_model_cnn.evaluate(X_test,y_test_one_hot)

print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')

23/23 [==============================] - 8s 364ms/step - loss: 0.2016 - categorical_accuracy: 0.9463
Loss: 0.20156486332416534
Accuracy: 0.9462517499923706


Save Model

In [17]:
# Save the model
model_filename = "travelahAlbertCNNFinal.h5"
path = f"drive/My Drive/BNGKT - Travelah CHATBOT/Model/{model_filename}" 
classifier_smallBERT_model_cnn.save(path)
print("Model saved as", model_filename)

Model saved as travelahAlbertCNNFinal.h5


Load model

In [18]:
import numpy as np
from tensorflow.keras.models import load_model

In [19]:
# Load the saved model
model_filename = "drive/My Drive/BNGKT - Travelah CHATBOT/Model/travelahAlbertCNNFinal.h5"
loaded_model = load_model(model_filename, custom_objects={'KerasLayer': hub.KerasLayer})
print("Model loaded from", model_filename)

Model loaded from drive/My Drive/BNGKT - Travelah CHATBOT/Model/travelahAlbertCNNFinal.h5


In [31]:
intent_mapping = df.groupby('intent_encoding')['intent'].first().to_dict()

user_utterance = "what should i prepare before i go"
user_utterance = np.array([user_utterance])

# Make the prediction
predictions = loaded_model.predict(user_utterance)
predicted_intent_index = np.argmax(predictions)

# Map the predicted intent index to intent label and response
predicted_intent_label = intent_mapping[str(predicted_intent_index)]
predicted_response = df.loc[df['intent'] == predicted_intent_label, 'response'].iloc[0]

print("Predicted Intent:", predicted_intent_label)
print("VELA:", predicted_response)

1/1 [==============================] - 0s 86ms/step
Predicted Intent: preparation_faq
VELA: Before jetting off to Bali, don't forget your passport, visa (if needed), travel insurance, vaccinations, research, bookings, currency exchange, transport plans, essentials, let others know, and stay connected on your adventure!


## JSON datasets

In [ ]:
def predict_intent_improved(input_text):
    with open('drive/My Drive/BNGKT - Travelah CHATBOT/Intent/intents.json', 'r') as file:
        intents_data = json.load(file)

    predictions = loaded_model.predict([input_text])
    predicted_intent_index = np.argmax(predictions)
    predicted_intent_probability = predictions[0][predicted_intent_index]

    for intent in intents_data['intent']:
        if intent['intent_encoding'] == str(predicted_intent_index):
            predicted_intent = intent['intent']
            predicted_responses = intent['responses'][0]
            break
    else:
        predicted_intent = ""
        predicted_responses = ""

    if predicted_intent_probability < 0.25:
        sorted_indices = np.argsort(predictions)[0][::-1]
        first_intent_index = sorted_indices[0]
        second_intent_index = sorted_indices[1]
        first_intent = intents_data['intent'][first_intent_index]['intent']
        second_intent = intents_data['intent'][second_intent_index]['intent']
        first_intent_probability = predictions[0][first_intent_index]
        second_intent_probability = predictions[0][second_intent_index]
        user_prompt = f"Is it what you mean? [{first_intent} (Probability: {first_intent_probability}), {second_intent} (Probability: {second_intent_probability})]"
        return user_prompt, predicted_responses, predicted_intent_probability

    return predicted_intent, predicted_responses, predicted_intent_probability


In [8]:
input_text = "i am in ubud, recommend me something"
user_prompt, responses, prob = predict_intent_improved(input_text)
print("Predicted Intent\t:", user_prompt)

if prob > 0.25:
  print("Predicted Responses\t:", responses)
  print("Probability\t\t:", prob)

NameError: ignored

In [ ]:
!pip freeze > requirements.txt
!pip list --format=freeze > requirements.txt

### Examples
```
http://127.0.0.1:5000/predict
```
#### **If intent is predicted correctly**
Input:
```
{
  "user_utterance": "who are you"
}
```
Output:

```
{
    "alt_intent_1": null,
    "alt_intent_2": null,
    "predicted_intent": "smalltalk_agent_acquaintance",
    "predicted_response": "Hey buddy! I'm Vela, your virtual tour guide for Bali. Ready to explore the beauty and awesomeness of this place? Ask away! Let's go on an epic adventure together!",
    "probability": 0.9692302942276001
}
```
#### **If intent is not predicted correctly**
Input:
```
{
  "user_utterance": "do you know MUSE band?"
}
```
Output:

```
{
    "alt_intent_1": "smalltalk_agent_age",
    "alt_intent_2": "smalltalk_agent_acquaintance",
    "predicted_intent": null,
    "predicted_response": "Sorry. I don't know what you just say. Do you mean one of this?",
    "probability": 0.23114833235740662
}
```


